# Web3 Community and Education sybil analysis

This notebooks looks for patterns of sybil bahevior in the Web3 Community and Education round.

This round was selected because during the grant review I flagged many projects from this round as suspicious.


We use sybil-scorer as well as new metrics to identify suspicious behavior.

We will then manually review the addresses using etherscan and DeBank to confirm our suspicions.

Then we try to see if any project is more affected by these flagged addresses in particular to back up our claims of suspicious project applications. This will also allow us to see if the sybil behavior is targeted towards a specific project.

In [1]:
import os
from pathlib import Path
import numpy as np

import pandas as pd

from sbdata.FlipsideApi import FlipsideApi

# Set path to data folder
current_dir = Path(os.getcwd())
PATH_TO_EXPORT = os.path.join(current_dir.parent.parent, 'tx_data', 'web3_community_and_education')
DATA_DIR = os.path.join(current_dir.parent.parent, 'data-regen-rangers')

# read the address from oss grant

api_key = os.environ['FLIPSIDE_API_KEY']
flipside_api = FlipsideApi(api_key, max_address=400)
PATH_TO_VOTES = os.path.join(DATA_DIR, "votes_baoki.csv")
PATH_TO_GRANTS = os.path.join(DATA_DIR, "all-allo-rounds.csv")
PATH_TO_PROJECTS = os.path.join(DATA_DIR, "projects_QmQurt.csv")

# load data
df_votes = pd.read_csv(PATH_TO_VOTES)
df_grants = pd.read_csv(PATH_TO_GRANTS)
df_application = pd.read_csv(PATH_TO_PROJECTS)
# Lowercase all addresses because flipside api return lowercase address
df_grants['Round ID'] = df_grants['Round ID'].str.lower()
str_columns_votes = ['id', 'transaction', 'projectId', 'roundId', 'voter', 'grantAddress']
df_votes[str_columns_votes] = df_votes[str_columns_votes].applymap(lambda x: x.lower())

str_columns_application = ['id', 'roundId', 'metadata.application.round', 'metadata.application.recipient']
df_application[str_columns_application] = df_application[str_columns_application].applymap(lambda x: str(x).lower())

round_id = df_grants[df_grants['Round name'] == 'Web3 Community and Education']['Round ID'].values[0]
array_unique_address = df_votes[df_votes['roundId'] == round_id]['voter'].unique()

array_unique_address = np.char.lower(array_unique_address.astype(str))


In [2]:
from sbutils import LoadData

# Load data
data_loader = LoadData.LoadData(PATH_TO_EXPORT)
df_tx = data_loader.create_df_tx('ethereum')

In [3]:
len(array_unique_address)

1023

In [4]:
df_tx.EOA.nunique()

1022

In [5]:
c = np.setxor1d(array_unique_address, df_tx.EOA.values)
c

array(['0x4a35674727c44cf4375d80c6171281ba2f764213'], dtype=object)

## Looking at a specific address donation

I don't know why that address is not retrieved by the API, but it is a voter and had some interesting patterns to explore. 
The address is: 0x4a35674727c44cf4375d80c6171281ba2f764213 voted to many projects by giving exactly the same amount to all of them. Let see:

In [6]:
suspicious_add = c[0]

In [7]:
df_vote_sus = df_votes[df_votes.voter== suspicious_add]
print(f'Number of votes: {len(df_vote_sus)}')
print(f'Number of unique projects: {df_vote_sus.projectId.nunique()}')
print(f'Number of unique grants: {df_vote_sus.grantAddress.nunique()}')

Number of votes: 48
Number of unique projects: 48
Number of unique grants: 47


Which project received the most votes from this address?

In [8]:
df_vote_sus['grantAddress'].value_counts().head(3)

grantAddress
0xb6e780438882f2daa11da0972807f4d12166af8b    2
0x486c853c41885d9c1edb57c423853fc4fbc60769    1
0xc36e4889a820bd8089a8ad226ee9d0c703aed314    1
Name: count, dtype: int64

In [9]:
# look at what is the project of the grant
suspicious_add = df_vote_sus['grantAddress'].value_counts().index[0]
df_application[df_application['metadata.application.recipient'] == suspicious_add]['metadata.application.project.title']


268    Impact DAOs Research + Podcast + Book : Impact...
337    Impact DAO Course + IRL/Online Events: Educati...
687    Impact DAOs Research + Podcast + Book : Impact...
Name: metadata.application.project.title, dtype: object

I find it very suspicious that this project "Impact DAO" applied for a grant with two subprojects. It would be interesting to explore how much common voter overlap there is between the two subprojects.

In [10]:
df_votes[df_votes['grantAddress'] == suspicious_add].groupby('projectId').count()

,id,transaction,blockNumber,applicationId,roundId,voter,grantAddress,token,amount,amountUSD,amountRoundToken,createdAt
projectId,,,,,,,,,,,,
0x3e7af275e7342f380ce8182e2f1e19ecbf32086974f53a386a8e1278016f6a50,19,19,19,19,19,19,19,19,19,19,19,19
0xa35e52635d40be090a49c587e58fab7410c30b9cff7ad5cc66b547d88cb93200,28,28,28,28,28,28,28,28,28,28,28,28


In [11]:
df_votes[df_votes['grantAddress'] == suspicious_add].groupby('projectId')['voter'].nunique()

projectId
0x3e7af275e7342f380ce8182e2f1e19ecbf32086974f53a386a8e1278016f6a50    18
0xa35e52635d40be090a49c587e58fab7410c30b9cff7ad5cc66b547d88cb93200    27
Name: voter, dtype: int64

In [12]:
df_votes[df_votes['grantAddress'] == suspicious_add].groupby('projectId')['amount'].describe()

,count,unique,top,freq
projectId,,,,
0x3e7af275e7342f380ce8182e2f1e19ecbf32086974f53a386a8e1278016f6a50,19,12,5000000000000000,3
0xa35e52635d40be090a49c587e58fab7410c30b9cff7ad5cc66b547d88cb93200,28,15,1000000000000000,6


The average donation is 1$ in both cases! 19 votes for 19$ and 28 votes for 28$.

This project is very suspicious and should proably be squelched.

## Computing legos booleans

In [13]:
from sblegos.TransactionAnalyser import TransactionAnalyser as txa
tx_analyser = txa(df_tx, df_address=pd.DataFrame(np.intersect1d(df_tx.EOA.unique(), array_unique_address)))

In [14]:
df_matching_address = pd.DataFrame(df_tx.EOA.unique(), columns=["address"])
df_matching_address.head(2)

,address
0,0x0010e3bce7e7d5890849fa2bb2681174f4352bc4
1,0x00173c4cb23e6d876fcb036ba954a2f9cfcafa19


Compute the boolean 

In [15]:
df_matching_address['seed_same_naive'] = df_matching_address.loc[:, 'address'].apply(lambda x : tx_analyser.has_same_seed_naive(x))
df_matching_address['seed_same'] = df_matching_address.loc[:, 'address'].apply(lambda x : tx_analyser.has_same_seed(x))
df_matching_address['seed_suspicious'] = df_matching_address.loc[:, 'seed_same_naive'].ne(df_matching_address.loc[:, 'seed_same'])
df_matching_address['less_5_tx'] = df_matching_address.loc[:, 'address'].apply(lambda x : tx_analyser.has_less_than_n_transactions(x, 5))
df_matching_address['less_10_tx'] = df_matching_address.loc[:, 'address'].apply(lambda x : tx_analyser.has_less_than_n_transactions(x, 10))
df_matching_address['interacted_other_ctbt'] = df_matching_address.loc[:, 'address'].apply(lambda x : tx_analyser.has_interacted_with_other_contributor(x))

In [16]:
print(f'Number of voters: {len(df_matching_address)}')

Number of voters: 1022


In [17]:
df_matching_address.sum()

address                  0x0010e3bce7e7d5890849fa2bb2681174f4352bc40x00...
seed_same_naive                                                        609
seed_same                                                              617
seed_suspicious                                                         14
less_5_tx                                                              209
less_10_tx                                                             283
interacted_other_ctbt                                                  211
dtype: object

### Investigating the boolean seed suspicious

In [18]:
df_matching_address[df_matching_address['seed_suspicious'] == True]

,address,seed_same_naive,seed_same,seed_suspicious,less_5_tx,less_10_tx,interacted_other_ctbt
19,0x04afa8026bff7e97443f4efd6f96c6f2ff6d4ec9,False,True,True,False,False,True
229,0x3a42f1dcc218b0ee48966954208c9c4acbc04d89,False,True,True,False,False,False
422,0x6b5918d8ef9094679f4b4e1bf397a66ea411b118,False,True,True,False,False,False
508,0x84f1cb27f0822135708aa879bbf04767af18019a,False,True,True,False,False,True
529,0x8a2ec4cc1d731bbace15fc5d234690a0788610a7,True,False,True,True,True,True
579,0x975a0e1bf2fdf3f4e5ba31b6c5774e73573b6b65,False,True,True,False,False,False
582,0x978eb534b26cb8749d352a2c94ec21e659e4248d,True,False,True,False,False,True
606,0x9d5c545d329454ab4ea9ff49a4e77985b9599232,False,True,True,False,False,True
673,0xac4ddaf8fbffba0f1e8c7619720335fd4f03eacd,True,False,True,False,False,True
774,0xc28064b875ae25f9a2ca28c08f116a5c26229f69,False,True,True,False,False,False


In [19]:
projects_voted = df_votes[df_votes['voter'] == '0xc28064b875ae25f9a2ca28c08f116a5c26229f69']
print(f'Number of votes {projects_voted.shape[0]} Number of projects voted: {projects_voted.grantAddress.nunique()}')
# Merge the project the user voted for and the projects 
projects_voted.merge(df_application, left_on='grantAddress', right_on='metadata.application.recipient', how='left').drop_duplicates(subset='grantAddress').loc[:, ['grantAddress', 'metadata.application.project.title', 'status', 'metadata.application.round']].reset_index(drop=True)

Number of votes 16 Number of projects voted: 16


,grantAddress,metadata.application.project.title,status,metadata.application.round
0,0x10666d9c6295e838d3b8b84ffcc97d62ef7e6120,Inverter Network - Fund and build in web 3 wit...,APPROVED,0x12bb5bbbfe596dbc489d209299b8302c3300fa40
1,0x11ee133a1408fe2d7c62296d7eb33f234b774503,dm3 protocol - the interoperability initiative,REJECTED,0x12bb5bbbfe596dbc489d209299b8302c3300fa40
2,0x7d658841f8ba93299970f6e765c2ce205f1e70dd,Loanshark,APPROVED,0x12bb5bbbfe596dbc489d209299b8302c3300fa40
3,0xb67fd6f4b908d702c2cf0e2b9d30d52d4ea5b2bc,ECHO,APPROVED,0x12bb5bbbfe596dbc489d209299b8302c3300fa40
4,0x713bc00d1df5c452f172c317d39eff71b771c163,Kakarot zkEVM,APPROVED,0xdf22a2c8f6ba9376ff17ee13e6154b784ee92094
5,0xdecf6615152ac768bfb688c4ef882e35debe08ac,Rhino Review - Ethereum Staking Journal,APPROVED,0xdf22a2c8f6ba9376ff17ee13e6154b784ee92094
6,0x187089b65520d2208ab93fb471c4970c29eaf929,Ape Framework,APPROVED,0xdf22a2c8f6ba9376ff17ee13e6154b784ee92094
7,0xb352bb4e2a4f27683435f153a259f1b207218b1b,eth.limo,REJECTED,0x12bb5bbbfe596dbc489d209299b8302c3300fa40
8,0xb7081fd06e7039d198d10a8b72b824e60c1b1e16,Otterscan,APPROVED,0xdf22a2c8f6ba9376ff17ee13e6154b784ee92094
9,0x52277e5cf8df8ef51d9bd37d8d7553a72d378bef,Trustless zkMafia,APPROVED,0x274554eb289004e15a7679123901b7f070dda0fa


Some of the projects he donated to are in the list of Rejected projects showing that this address is indeed a sybil. And may have contributed to ohter fraudulent projects.

- Pulsar is not very active and is forked code for the most part
- Fusion not very active on github but has a lot of activity on twitter 
- Share suspicious Github with no activity, twitter does not exists: suspicious

Other projects are ok

In [20]:
projects_voted = df_votes[df_votes['voter'] == '0x61ffe691821291d02e9ba5d33098adcee71a3a17']
print(f'Number of votes {projects_voted.shape[0]} Number of projects voted: {projects_voted.grantAddress.nunique()}')
# Merge the project the user voted for and the projects 
projects_voted.merge(df_application, left_on='grantAddress', right_on='metadata.application.recipient', how='left').drop_duplicates(subset='grantAddress').loc[:, ['grantAddress', 'metadata.application.project.title']].reset_index(drop=True)

Number of votes 3 Number of projects voted: 3


,grantAddress,metadata.application.project.title
0,0x5514f4a2bc7194664b12a48e238876aa53140350,Voting Contracts
1,0x53390590476dc98860316e4b46bb9842af55efc4,DAppNode
2,0x4cd6b4503d02973d78cff59c9c56f5f378688274,Mechanism Institute: Democratizing Cryptoecono...


In [21]:
projects_voted = df_votes[df_votes['voter'] == '0xe51200a4d161935fc311ed8a0401feb1abf20e3a']
print(f'Number of votes {projects_voted.shape[0]} Number of projects voted: {projects_voted.grantAddress.nunique()}')
# Merge the project the user voted for and the projects 
projects_voted.merge(df_application, left_on='grantAddress', right_on='metadata.application.recipient', how='left').drop_duplicates(subset='grantAddress').loc[:, ['grantAddress', 'metadata.application.project.title']].reset_index(drop=True)

Number of votes 2 Number of projects voted: 2


,grantAddress,metadata.application.project.title
0,0x2f6c8f867df4e49e4fb24741c414f315e266c21b,Ethereum Lima
1,0xba97f51b1b097ce1ea43cf941cd62dfbcd70cae3,ETH Kipu


The boolean seed suspicious is not relevant for that round we will not use it for the analysis.

### Computing the new dex interaction score
It was investigated in another notebook 

In [22]:
def get_interacted_address(from_address, to_address, address):
    if from_address == address:
        return to_address
    else:
        return from_address

def count_interaction_with_any(tx_analyser, address, array_address):
    """
    Return an integer of the number of interactions with the addresses in the array_address
    Parameters
    ----------
    address : str
        The address to check

    Returns
    -------
    count_interaction_with_any : int
        The number of interactions with the addresses in the array_address
    """
    tx_analyser.set_group_by_sorted_EOA()

    df = tx_analyser.gb_EOA_sorted.get_group(address)
    address_interacted = df.apply(lambda x: get_interacted_address(x['from_address'], x['to_address'], address), axis=1)
    tx_boolean_interacted = address_interacted.isin(array_address)
    return tx_boolean_interacted.sum()

def has_interacted_with_any(tx_analyser, address, array_address):
    """
    Return a boolean whether the address has interacted with any address in the array_address
    Parameters
    ----------
    address : str
        The address to check

    Returns
    -------
    has_interacted_with_any : bool
        True if the address has interacted with one or more of the addresses in the array_address
    """
    count_interaction_with_any = count_interaction_with_any(tx_analyser, address, array_address)
    return count_interaction_with_any > 0

In [23]:
label_query = '''
SELECT ADDRESS, CREATOR, LABEL_TYPE, ADDRESS_NAME, PROJECT_NAME
FROM crosschain.core.address_labels 
WHERE BLOCKCHAIN='ethereum'
AND LABEL_SUBTYPE = 'pool' 
;'''
df_label = flipside_api.execute_query(label_query)

In [24]:
# extract all the pool addresses
array_pool_address = df_label['address'].unique()

In [25]:
tx_analyser.set_group_by_sorted_EOA()

In [26]:
# Compute the number of interactions with any of the pools for each address
df_matching_address['count_interaction_with_pool'] = df_matching_address['address'].apply(lambda x: count_interaction_with_any(tx_analyser, x, array_pool_address))

In [27]:
(df_matching_address['count_interaction_with_pool'] > 0).sum() / len(df_matching_address)

0.32093933463796476

In [28]:
label_query = '''
SELECT DISTINCT(LABEL_SUBTYPE)
FROM crosschain.core.address_labels 
WHERE BLOCKCHAIN='ethereum'
;'''
df_distinct_labels = flipside_api.execute_query(label_query)

In [29]:
df_distinct_labels.label_subtype.unique()

array(['strategy', 'chadmin', 'router', 'multisig', 'airdrop_contract',
       'nf_token_contract', 'token_distribution', 'nf_position_manager',
       'donation_address', 'mining_pool', 'deposit_wallet',
       'staking_contract', 'pool', 'token_contract', 'bridge',
       'mint_burn', 'mint_contract', 'contract_deployer', 'marketplace',
       'foundation', 'vault', 'voting', 'cold_wallet', 'reserve',
       'swap_router', 'hot_wallet', 'toxic', 'fee_wallet', 'governance',
       'general_contract', 'swap_contract', 'distributor_cex', 'escrow',
       'rewards', 'dao', 'treasury', 'token_sale', 'oracle',
       'aggregator_contract'], dtype=object)

From these tags lets flag any address that have interacted with a toxic wallet

In [30]:
label_query = '''
SELECT ADDRESS, CREATOR, LABEL_TYPE, ADDRESS_NAME, PROJECT_NAME
FROM crosschain.core.address_labels 
WHERE BLOCKCHAIN='ethereum'
AND LABEL_SUBTYPE = 'toxic'
;'''
df_toxic = flipside_api.execute_query(label_query)

In [31]:
df_toxic.shape

(4857, 6)

In [32]:
# Compute the number of interactions with any of the scam for each address
df_matching_address['count_interaction_with_toxic'] = df_matching_address['address'].apply(lambda x: count_interaction_with_any(tx_analyser, x, df_toxic['address'].unique()))

In [33]:
print(f'Percentage of addresses that have interacted with a toxic address: {int((df_matching_address["count_interaction_with_toxic"] > 0).sum() / len(df_matching_address) *100)}%')

Percentage of addresses that have interacted with a toxic address: 1%


In [34]:
tag_query = '''
SELECT DISTINCT(TAG_TYPE)
FROM crosschain.core.address_tags 
WHERE BLOCKCHAIN='ethereum'
;'''
df_distinct_tags = flipside_api.execute_query(tag_query)

In [35]:
df_distinct_tags.tag_type.values

array(['contract', 'Balancer Delegates', 'wallet', 'activity', 'cex',
       'chainlink oracle', 'Aave Delegates', 'NFT', 'nft', 'dex'],
      dtype=object)

I found the tag_name "airdrop master" could be interesting 

In [36]:
query_airdrop_master = '''
SELECT BLOCKCHAIN, CREATOR, ADDRESS, TAG_NAME
FROM crosschain.core.address_tags 
WHERE BLOCKCHAIN='ethereum'
AND TAG_NAME = 'airdrop master'
;
'''
df_airdrop_master = flipside_api.execute_query(query_airdrop_master)

In [37]:
# Compute the number of interactions with any of the aidrop for each address
df_matching_address['count_interaction_with_airdrop_m'] = df_matching_address['address'].apply(lambda x: count_interaction_with_any(tx_analyser, x, df_airdrop_master['address'].unique()))
print(f'Percentage of addresses that interacted with airdrop master: {int((df_matching_address["count_interaction_with_airdrop_m"] > 0).sum() / len(df_matching_address) * 100)}%')

Percentage of addresses that interacted with airdrop master: 19%


In [38]:
# Boolean whether the address is a aidrop master
df_matching_address['is_airdrop_master'] = df_matching_address['address'].apply(lambda x: x in df_airdrop_master['address'].unique()) 
print(f'Percentage of addresses that are airdrop master: {int((df_matching_address["is_airdrop_master"]).sum() / len(df_matching_address) * 100)}%')

Percentage of addresses that are airdrop master: 6%


In [39]:
sql_query_tornado = '''
SELECT DISTINCT PROJECT_NAME, ADDRESS
FROM crosschain.core.address_labels 
WHERE BLOCKCHAIN='ethereum'
AND PROJECT_NAME LIKE '%tornado%'
;
'''
df_tornado = flipside_api.execute_query(sql_query_tornado)

In [40]:
# Count the number of interactions with tornado
df_matching_address['count_interaction_with_tornado'] = df_matching_address['address'].apply(lambda x: count_interaction_with_any(tx_analyser, x, df_tornado['address'].unique()))
print(f'Percentage of addresses that interacted with tornado: {int((df_matching_address["count_interaction_with_tornado"] > 0).sum() / len(df_matching_address) * 100)}%')

Percentage of addresses that interacted with tornado: 3%


In [41]:
# Count the number of time the address interatec with disperse contract: '0xD152f549545093347A162Dce210e7293f1452150'
df_matching_address['count_interaction_with_disperse'] = df_matching_address['address'].apply(lambda x: count_interaction_with_any(tx_analyser, x, [str.lower('0xD152f549545093347A162Dce210e7293f1452150')]))
print(f'Percentage of addresses that interacted with disperse: {int((df_matching_address["count_interaction_with_disperse"] > 0).sum() / len(df_matching_address) * 100)}%')

Percentage of addresses that interacted with disperse: 1%


In [42]:
df_matching_address.describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.9])

,count_interaction_with_pool,count_interaction_with_toxic,count_interaction_with_airdrop_m,count_interaction_with_tornado,count_interaction_with_disperse
count,1022.000000,1022.00000,1022.000000,1022.000000,1022.000000
mean,6.946184,0.02544,1.773973,0.103718,0.078278
std,38.091056,0.23281,7.696927,0.830966,0.901483
min,0.000000,0.00000,0.000000,0.000000,0.000000
10%,0.000000,0.00000,0.000000,0.000000,0.000000
25%,0.000000,0.00000,0.000000,0.000000,0.000000
50%,0.000000,0.00000,0.000000,0.000000,0.000000
75%,2.000000,0.00000,0.000000,0.000000,0.000000
90%,12.000000,0.00000,3.000000,0.000000,0.000000
max,742.000000,5.00000,96.000000,18.000000,16.000000


In [43]:
df_matching_address['has_interaction_toxic'] = df_matching_address['count_interaction_with_toxic'] > 0
df_matching_address['has_no_pool_interaction'] = df_matching_address['count_interaction_with_pool'] < 6
df_matching_address['has_interaction_airdrop_m'] = df_matching_address['count_interaction_with_airdrop_m'] > 0
df_matching_address['has_interaction_tornado'] = df_matching_address['count_interaction_with_tornado'] > 0
df_matching_address['has_interaction_disperse'] = df_matching_address['count_interaction_with_disperse'] > 0

In [44]:
boolean_features = ['has_interaction_toxic', 'has_no_pool_interaction', 'has_interaction_airdrop_m', 'has_interaction_tornado', 'has_interaction_disperse', 'is_airdrop_master', 'interacted_other_ctbt', 'less_10_tx', 'less_5_tx']

In [45]:
df_matching_address[boolean_features].sum() 

has_interaction_toxic         18
has_no_pool_interaction      870
has_interaction_airdrop_m    200
has_interaction_tornado       32
has_interaction_disperse      17
is_airdrop_master             70
interacted_other_ctbt        211
less_10_tx                   283
less_5_tx                    209
dtype: int64

In [46]:
(df_matching_address[boolean_features].sum(axis=1) > 2).sum()

305

In [47]:
df_matching_address['count_flags'] = df_matching_address[boolean_features].sum(axis=1)

In [48]:
df_matching_address['suspicious_1'] = df_matching_address['count_flags'] > 2

In [49]:
df_suspicious_1 = df_matching_address[df_matching_address['suspicious_1'] == True]

### Investigating the grants receiving the most votes from the flagged addresses

In [50]:
df_vote_sus1 = df_votes[df_votes['voter'].isin(df_suspicious_1['address'])]

In [51]:
print(f'Number of votes {df_vote_sus1.shape[0]} Number of projects voted: {df_vote_sus1.grantAddress.nunique()}')
# Merge the project the user voted for and the projects 
gr_sus = df_vote_sus1['grantAddress'].value_counts().reset_index().merge(df_application, left_on='grantAddress', right_on='metadata.application.recipient', how='left').drop_duplicates(subset='grantAddress').loc[:, ['grantAddress', 'metadata.application.project.title', 'count', 'roundId', 'status']].reset_index(drop=True)

Number of votes 1304 Number of projects voted: 395


In [52]:
gr_sus.head(30)

,grantAddress,metadata.application.project.title,count,roundId,status
0,0xc36e4889a820bd8089a8ad226ee9d0c703aed314,Crypto Babes Club,173,0xaa40e2e5c8df03d792a52b5458959c320f86ca18,APPROVED
1,0x4adc8cc149a03f44386bee80bab36f9e8022b195,Unitap,17,0xaa40e2e5c8df03d792a52b5458959c320f86ca18,APPROVED
2,0x8110d1d04ac316fdcace8f24fd60c86b810ab15a,Commons Stack,14,0x12bb5bbbfe596dbc489d209299b8302c3300fa40,APPROVED
3,0x18aa467e40e1defb1956708830a343c1d01d3d7c,JediSwap,12,0x12bb5bbbfe596dbc489d209299b8302c3300fa40,APPROVED
4,0x4d9339dd97db55e3b9bcbe65de39ff9c04d1c2cd,Giveth,11,0x12bb5bbbfe596dbc489d209299b8302c3300fa40,APPROVED
5,0xda3dddae8119644f8be18c7afd16850b02a4c841,ETH Venezuela Community Projects,11,0xaa40e2e5c8df03d792a52b5458959c320f86ca18,APPROVED
6,0x3a5bd1e37b099ae3386d13947b6a90d97675e5e3,Lenster,11,0x12bb5bbbfe596dbc489d209299b8302c3300fa40,APPROVED
7,0x1dd8ec683e8b53c57ba5a01c5d2f866a3fbd3ce8,Help Protect the Internet Archive,10,0xaa40e2e5c8df03d792a52b5458959c320f86ca18,APPROVED
8,0xd8eab52bccf6307a4579ab868b42eee0eb5492ed,Ethereum Uruguay,10,0xaa40e2e5c8df03d792a52b5458959c320f86ca18,APPROVED
9,0x57e6cf96a2e99a3fdaa79acc3a14fc9be1eabc03,The Neptune Project,10,0xaa40e2e5c8df03d792a52b5458959c320f86ca18,APPROVED


Upchain twitter handle does not exists but the repo is very active Suspicious?

- Seoul bound is either very new or suspicious
- Gravity DAO old project not sure it is still active

Other projects are not suspicious

### Look at the addresses with many flags

In [53]:
df_matching_address[boolean_features].sum()

has_interaction_toxic         18
has_no_pool_interaction      870
has_interaction_airdrop_m    200
has_interaction_tornado       32
has_interaction_disperse      17
is_airdrop_master             70
interacted_other_ctbt        211
less_10_tx                   283
less_5_tx                    209
dtype: int64

We are going to review the addresses that have at least 1 flag:
- has_intercation_toxic 
- has_interaction_tornado
- has_interaction_disperse
- has_interaction_airdrop_master
- is airdrop master

In [54]:
interaction_bool = ['has_interaction_toxic', 'has_interaction_airdrop_m', 'has_interaction_tornado', 'has_interaction_disperse', 'is_airdrop_master']

In [55]:
df_interact_sus = df_matching_address[df_matching_address[interaction_bool].sum(axis=1) > 0]
print(f'Number of addresses that interacted with a suspicious contract or address: {df_interact_sus.shape[0]}')

Number of addresses that interacted with a suspicious contract or address: 226


In [56]:
df_vote_interact_sus = df_votes[df_votes['voter'].isin(df_interact_sus['address'])]

In [57]:
print(f'Number of votes {df_vote_sus1.shape[0]} Number of projects voted: {df_vote_sus1.grantAddress.nunique()}')
# Merge the project the user voted for and the projects 
gr_sus = df_vote_interact_sus['grantAddress'].value_counts().reset_index().merge(df_application, left_on='grantAddress', right_on='metadata.application.recipient', how='left').drop_duplicates(subset='grantAddress').loc[:, ['grantAddress', 'metadata.application.project.title', 'count', 'roundId', 'status']].reset_index(drop=True)

Number of votes 1304 Number of projects voted: 395


In [58]:
gr_sus.sort_values(by='count', ascending=False).head(30)

,grantAddress,metadata.application.project.title,count,roundId,status
0,0x3a5bd1e37b099ae3386d13947b6a90d97675e5e3,Lenster,35,0x12bb5bbbfe596dbc489d209299b8302c3300fa40,APPROVED
1,0x18aa467e40e1defb1956708830a343c1d01d3d7c,JediSwap,29,0x12bb5bbbfe596dbc489d209299b8302c3300fa40,APPROVED
2,0x08a3c2a819e3de7aca384c798269b3ce1cd0e437,DefiLlama,26,0x12bb5bbbfe596dbc489d209299b8302c3300fa40,APPROVED
3,0x4d9339dd97db55e3b9bcbe65de39ff9c04d1c2cd,Giveth,26,0x12bb5bbbfe596dbc489d209299b8302c3300fa40,APPROVED
4,0xe126b3e5d052f1f575828f61feba4f4f2603652a,Revoke.cash - Helping you stay safe in web3,26,0x12bb5bbbfe596dbc489d209299b8302c3300fa40,APPROVED
5,0x8110d1d04ac316fdcace8f24fd60c86b810ab15a,Commons Stack,25,0x12bb5bbbfe596dbc489d209299b8302c3300fa40,APPROVED
6,0x99b36fdbc582d113af36a21eba06bfeab7b9be12,Taho - Open Source and Community Owned Wallet,23,0x12bb5bbbfe596dbc489d209299b8302c3300fa40,APPROVED
7,0x4b8810b079eb22ecf2d1f75e08e0abbd6fd87dbf,BrightID 🔆 Universal Proof of Uniqueness,22,0x12bb5bbbfe596dbc489d209299b8302c3300fa40,APPROVED
8,0x4adc8cc149a03f44386bee80bab36f9e8022b195,Unitap,21,0xaa40e2e5c8df03d792a52b5458959c320f86ca18,APPROVED
11,0xb352bb4e2a4f27683435f153a259f1b207218b1b,eth.limo,20,0x12bb5bbbfe596dbc489d209299b8302c3300fa40,REJECTED


No projects look suspicious in the top 30 by contribution

### Investigating more in details the type of donations made by the flagged addresses

In [59]:
# look at the average donation in USD
gb_donation_usd = df_vote_interact_sus.groupby('voter').describe()['amountUSD']

In [60]:
gb_donation_usd[gb_donation_usd['count'] == 1].sort_values(by='count', ascending=True)

,count,mean,std,min,25%,50%,75%,max
voter,,,,,,,,
0x008c9f31533dd2a327d0ff28b6fe87194de1e3d3,1.0,5.997155,NaN,5.997155,5.997155,5.997155,5.997155,5.997155
0xa0839d50d601c0ce86677a03739f802fe614ba22,1.0,19.997351,NaN,19.997351,19.997351,19.997351,19.997351,19.997351
0xa009881d1ecddb62a2e15f0ef19c482dd2df1c3a,1.0,92.636609,NaN,92.636609,92.636609,92.636609,92.636609,92.636609
0x9b278aaef429cc4728e594f8fa0fbfe78e72cc26,1.0,1.138289,NaN,1.138289,1.138289,1.138289,1.138289,1.138289
0x921f80499a00ac6e95aae0daa411d338f41d5da2,1.0,9.528923,NaN,9.528923,9.528923,9.528923,9.528923,9.528923
...,...,...,...,...,...,...,...,...
0x40f9bf922c23c43acdad71ab4425280c0ffbd697,1.0,200.691627,NaN,200.691627,200.691627,200.691627,200.691627,200.691627
0x3ffc07ed6a65e379e39bef93e713ee3ff77a56e4,1.0,36.972994,NaN,36.972994,36.972994,36.972994,36.972994,36.972994
0x3e48830bf4415311d13c3c89a0e315ee7398d114,1.0,3.792925,NaN,3.792925,3.792925,3.792925,3.792925,3.792925


This is interesting lets look at which project they gave

Nonetheless some donations are very similar in the amount donated lets try to dig into them

In [61]:
# inparticular address that donated close to 9.5$
same_amount = gb_donation_usd[np.logical_and(gb_donation_usd['count'] == 1, np.floor(gb_donation_usd['mean']) == 9)].sort_values(by='count', ascending=True)
same_amount

,count,mean,std,min,25%,50%,75%,max
voter,,,,,,,,
0x2e3e22f9f4d2de3dbdee9cbf350e22afb1c1d6b9,1.0,9.515146,NaN,9.515146,9.515146,9.515146,9.515146,9.515146
0x588f6ccb39f3d474c5b3736483589e2bbc8e0847,1.0,9.281587,NaN,9.281587,9.281587,9.281587,9.281587,9.281587
0x70b224998167a500ef39ed91e35ad133d996e744,1.0,9.494141,NaN,9.494141,9.494141,9.494141,9.494141,9.494141
0x71f84aa585e1eda8852dad8dff3a69d114366dba,1.0,9.734560,NaN,9.734560,9.734560,9.734560,9.734560,9.734560
0x7267db34b6072080923f92e626c2fc5da91ff25b,1.0,9.219747,NaN,9.219747,9.219747,9.219747,9.219747,9.219747
0x745339df47cb4186be2fc573e81d50e598b99aef,1.0,9.399484,NaN,9.399484,9.399484,9.399484,9.399484,9.399484
0x921f80499a00ac6e95aae0daa411d338f41d5da2,1.0,9.528923,NaN,9.528923,9.528923,9.528923,9.528923,9.528923
0xa34faa5b9b9e29c191e163a520edeea93a574ba3,1.0,9.497355,NaN,9.497355,9.497355,9.497355,9.497355,9.497355
0xae65dfac43c1dd0b4e4976128d654adf597f756c,1.0,9.556308,NaN,9.556308,9.556308,9.556308,9.556308,9.556308


In [62]:
def get_grant_title(address, df_application, df_votes):
    project_address = df_votes[df_votes['voter'] == address]['grantAddress'].values[0]
    return df_application[df_application['metadata.application.recipient'] == project_address]['metadata.application.project.title'].values[0]

In [63]:
for address in same_amount.index:
    print(f'Address: {address} Project: {get_grant_title(address, df_application, df_votes)}')

Address: 0x2e3e22f9f4d2de3dbdee9cbf350e22afb1c1d6b9 Project: Unitap
Address: 0x588f6ccb39f3d474c5b3736483589e2bbc8e0847 Project: Mechanism Institute: Democratizing Cryptoeconomics
Address: 0x70b224998167a500ef39ed91e35ad133d996e744 Project: BanklessDAO Projects
Address: 0x71f84aa585e1eda8852dad8dff3a69d114366dba Project: Web3TalentFair
Address: 0x7267db34b6072080923f92e626c2fc5da91ff25b Project: Unitap
Address: 0x745339df47cb4186be2fc573e81d50e598b99aef Project: GreenPill Global 
Address: 0x921f80499a00ac6e95aae0daa411d338f41d5da2 Project: JobStash
Address: 0xa34faa5b9b9e29c191e163a520edeea93a574ba3 Project: BanklessDAO Projects
Address: 0xae65dfac43c1dd0b4e4976128d654adf597f756c Project: Web3ForGood
Address: 0xaee2e6432db54a6f3770d57f1108cd326f2b224f Project: BanklessDAO Projects
Address: 0xbb49ccec023dc369844eec3190503853b1380606 Project: BanklessDAO Projects
Address: 0xe4c140a67a3a95913a6e2fcfc6d6434d94c07641 Project: ReFi Japan
Address: 0xe8fb09228d1373f931007ca7894a08344b80901c Pr

This analysis is pointing to a suspicion for the project "BanklessDAO Projects" and "Unitap"

However the project "BanklessDAO Projects" is well known.

My guess is that there is a lot of collusion within these addresses and there donation should not be matched. Especially knowing some of these addresses are flagged. Let's verify it.

In [64]:
df_matching_address[df_matching_address['address'].isin(same_amount.index)]

,address,seed_same_naive,seed_same,seed_suspicious,less_5_tx,less_10_tx,interacted_other_ctbt,count_interaction_with_pool,count_interaction_with_toxic,count_interaction_with_airdrop_m,is_airdrop_master,count_interaction_with_tornado,count_interaction_with_disperse,has_interaction_toxic,has_no_pool_interaction,has_interaction_airdrop_m,has_interaction_tornado,has_interaction_disperse,count_flags,suspicious_1
172,0x2e3e22f9f4d2de3dbdee9cbf350e22afb1c1d6b9,False,False,False,False,False,True,5,0,12,True,0,0,False,True,True,False,False,4,True
348,0x588f6ccb39f3d474c5b3736483589e2bbc8e0847,False,False,False,False,False,False,14,0,0,False,1,0,False,False,False,True,False,1,False
439,0x70b224998167a500ef39ed91e35ad133d996e744,False,False,False,False,False,True,2,0,4,True,0,0,False,True,True,False,False,4,True
445,0x71f84aa585e1eda8852dad8dff3a69d114366dba,False,False,False,False,False,False,8,0,1,True,0,0,False,False,True,False,False,2,False
447,0x7267db34b6072080923f92e626c2fc5da91ff25b,True,True,False,False,False,True,3,0,2,False,0,0,False,True,True,False,False,3,True
455,0x745339df47cb4186be2fc573e81d50e598b99aef,True,True,False,False,False,False,0,0,1,False,0,0,False,True,True,False,False,2,False
556,0x921f80499a00ac6e95aae0daa411d338f41d5da2,False,False,False,False,False,True,3,0,0,False,1,0,False,True,False,True,False,3,True
636,0xa34faa5b9b9e29c191e163a520edeea93a574ba3,False,False,False,False,False,True,3,0,42,False,1,0,False,True,True,True,False,4,True
685,0xae65dfac43c1dd0b4e4976128d654adf597f756c,False,False,False,False,False,True,0,0,5,False,0,0,False,True,True,False,False,3,True
687,0xaee2e6432db54a6f3770d57f1108cd326f2b224f,False,False,False,False,False,True,2,0,19,False,0,0,False,True,True,False,False,3,True


In [65]:
same_amount_tx = df_tx[np.logical_or(df_tx['from_address'].isin(same_amount.index), df_tx['to_address'].isin(same_amount.index))]
txa_same_amount = txa(same_amount_tx, pd.DataFrame(same_amount.index, columns=['address']))

All true values mean that the wallet are connected between themselves in that small cluster of addresses

In [66]:
same_amount.reset_index()['voter'].apply(lambda x : txa_same_amount.has_interacted_with_other_contributor(x)).values

array([ True, False,  True, False,  True, False,  True,  True,  True,
        True,  True, False,  True, False])

In [67]:
# explore other suspicious addresses
add_sus = gb_donation_usd[np.logical_and(gb_donation_usd['count'] == 1, gb_donation_usd['mean'] < 25)].index.values
len(add_sus)

45

In [68]:
df_1_donation = df_vote_interact_sus[df_vote_interact_sus['voter'].isin(add_sus)]

In [69]:
print(f'Number of votes {df_1_donation.shape[0]} Number of projects voted: {df_1_donation.grantAddress.nunique()}')
# Merge the project the user voted for and the projects 
gr_sus = df_1_donation['grantAddress'].value_counts().reset_index().merge(df_application, left_on='grantAddress', right_on='metadata.application.recipient', how='left').drop_duplicates(subset='grantAddress').loc[:, ['grantAddress', 'metadata.application.project.title', 'count', 'roundId', 'status']].reset_index(drop=True)
gr_sus.sort_values(by='count', ascending=False)

Number of votes 45 Number of projects voted: 28


,grantAddress,metadata.application.project.title,count,roundId,status
0,0x4adc8cc149a03f44386bee80bab36f9e8022b195,Unitap,9,0xaa40e2e5c8df03d792a52b5458959c320f86ca18,APPROVED
1,0xf26d1bb347a59f6c283c53156519cc1b1abaca51,BanklessDAO Projects,4,0xaa40e2e5c8df03d792a52b5458959c320f86ca18,APPROVED
2,0x4cd6b4503d02973d78cff59c9c56f5f378688274,Mechanism Institute: Democratizing Cryptoecono...,3,0xaa40e2e5c8df03d792a52b5458959c320f86ca18,APPROVED
3,0xaa163c47065c22d17ed1c47e3e244337d2056c17,JobStash,2,0xaa40e2e5c8df03d792a52b5458959c320f86ca18,APPROVED
4,0x8efef51d19ef3844c00076ab9d02847b9c70f94a,Pentacle,2,0xaa40e2e5c8df03d792a52b5458959c320f86ca18,APPROVED
5,0xdb587c215626cc38c04f9e283e159f3907791baf,xBound,2,0xaa40e2e5c8df03d792a52b5458959c320f86ca18,APPROVED
6,0xd6823f807c45efdc56c9ae8db0226ca10af6e8ab,登链社区(Upchain),2,0xaa40e2e5c8df03d792a52b5458959c320f86ca18,APPROVED
22,0x98714debf8c3114cffbdf04bb49c2e24c899c932,Open Town,1,0xaa40e2e5c8df03d792a52b5458959c320f86ca18,APPROVED
19,0x486c853c41885d9c1edb57c423853fc4fbc60769,Blu3 Global,1,0xaa40e2e5c8df03d792a52b5458959c320f86ca18,APPROVED
20,0x2ec655969160ab38d308140f3817a0e3be0ca9f2,KultureCity F THE NEVERS,1,0xaa40e2e5c8df03d792a52b5458959c320f86ca18,APPROVED


By reviewing manually I did not find that any of these projects were suspicious.

In [70]:
df_matching_address[df_matching_address['has_interaction_toxic']]

,address,seed_same_naive,seed_same,seed_suspicious,less_5_tx,less_10_tx,interacted_other_ctbt,count_interaction_with_pool,count_interaction_with_toxic,count_interaction_with_airdrop_m,is_airdrop_master,count_interaction_with_tornado,count_interaction_with_disperse,has_interaction_toxic,has_no_pool_interaction,has_interaction_airdrop_m,has_interaction_tornado,has_interaction_disperse,count_flags,suspicious_1
36,0x0a251df99a88a20a93876205fb7f5faf2e85a481,False,False,False,False,False,False,106,1,10,True,1,0,True,False,True,True,False,4,True
45,0x0bb602f88bf886282ff69d4cec937cc2a7d9e19a,True,True,False,False,False,False,15,5,0,False,0,0,True,False,False,False,False,1,False
158,0x293c173b0ae90cd27c349cd8892efa625a98dbae,False,False,False,False,False,True,0,1,3,False,0,0,True,True,True,False,False,4,True
308,0x504c11bdbe6e29b46e23e9a15d9c8d2e2e795709,False,False,False,False,False,False,34,1,11,False,4,2,True,False,True,True,True,4,True
356,0x5a930b098ed8d58dd4590577af85a8e864a8f6fe,True,True,False,False,False,True,0,1,0,False,0,0,True,True,False,False,False,3,True
427,0x6d526f6b4c86fbdc8e359e6bef4cd6a42acea2d7,True,True,False,False,False,False,6,1,3,True,0,0,True,False,True,False,False,3,True
460,0x767a60f295aedd958932088f9cd6a4951d8739b6,False,False,False,False,False,False,78,1,8,True,0,0,True,False,True,False,False,3,True
481,0x7bc1d6e8150d4c96917901c26c922dd64654b3c3,True,True,False,False,False,False,64,1,0,False,0,0,True,False,False,False,False,1,False
510,0x85274906f537e0aa3823855bd6a0e374c771d19b,False,False,False,False,False,True,4,1,3,False,0,0,True,True,True,False,False,4,True
563,0x93907de38066d70109935732757b625d636e47b6,True,True,False,False,False,True,16,1,0,False,0,0,True,False,False,False,False,2,False


In [71]:
df_matching_address[np.logical_and(np.logical_and(df_matching_address['has_interaction_airdrop_m'], df_matching_address['is_airdrop_master']), df_matching_address['has_interaction_disperse'])]

,address,seed_same_naive,seed_same,seed_suspicious,less_5_tx,less_10_tx,interacted_other_ctbt,count_interaction_with_pool,count_interaction_with_toxic,count_interaction_with_airdrop_m,is_airdrop_master,count_interaction_with_tornado,count_interaction_with_disperse,has_interaction_toxic,has_no_pool_interaction,has_interaction_airdrop_m,has_interaction_tornado,has_interaction_disperse,count_flags,suspicious_1
319,0x52432473144056fe91aeae9240ad21e6b8213440,False,False,False,False,False,True,3,0,4,True,0,1,False,True,True,False,True,5,True
503,0x839395e20bbb182fa440d08f850e6c7a8f6f0780,False,False,False,False,False,True,245,0,96,True,8,1,False,False,True,True,True,5,True
561,0x9325564ade7683706107685cf1993678b1163261,False,False,False,False,False,False,59,0,10,True,18,16,False,False,True,True,True,4,True
696,0xb08f95dbc639621dbaf48a472ae8fce0f6f56a6e,True,True,False,False,False,True,56,0,40,True,0,3,False,False,True,False,True,4,True
857,0xd6f5646d9e7fbee7cc907eb8e12dafa5378431e6,False,False,False,False,False,True,14,0,40,True,1,1,False,False,True,True,True,5,True
1002,0xfb40932271fc9db9dbf048e80697e2da4aa57250,False,False,False,False,False,True,76,1,68,True,0,1,True,False,True,False,True,5,True


Amost every airdrop master has interacted with another aidrop master. This is very suspicious. It means they are very connected to each other. These addresses are probably controlled by the same person or a group of person and thus their donation should not be matched.

In [72]:
df_suspicious_1.sort_values(by='count_flags', ascending=False).head(20)

,address,seed_same_naive,seed_same,seed_suspicious,less_5_tx,less_10_tx,interacted_other_ctbt,count_interaction_with_pool,count_interaction_with_toxic,count_interaction_with_airdrop_m,is_airdrop_master,count_interaction_with_tornado,count_interaction_with_disperse,has_interaction_toxic,has_no_pool_interaction,has_interaction_airdrop_m,has_interaction_tornado,has_interaction_disperse,count_flags,suspicious_1
319,0x52432473144056fe91aeae9240ad21e6b8213440,False,False,False,False,False,True,3,0,4,True,0,1,False,True,True,False,True,5,True
503,0x839395e20bbb182fa440d08f850e6c7a8f6f0780,False,False,False,False,False,True,245,0,96,True,8,1,False,False,True,True,True,5,True
416,0x693edbcf118ec982f5a8101498b6c789470b0b89,False,False,False,False,False,True,1,0,3,True,2,0,False,True,True,True,False,5,True
449,0x7307174ed422e37cb5254b944ffa762907f0b41b,False,False,False,True,True,True,0,0,1,False,0,0,False,True,True,False,False,5,True
453,0x74160cb9d7900c65e9f716bf3fde36c0ec335a78,True,True,False,True,True,True,0,0,1,False,0,0,False,True,True,False,False,5,True
857,0xd6f5646d9e7fbee7cc907eb8e12dafa5378431e6,False,False,False,False,False,True,14,0,40,True,1,1,False,False,True,True,True,5,True
1002,0xfb40932271fc9db9dbf048e80697e2da4aa57250,False,False,False,False,False,True,76,1,68,True,0,1,True,False,True,False,True,5,True
561,0x9325564ade7683706107685cf1993678b1163261,False,False,False,False,False,False,59,0,10,True,18,16,False,False,True,True,True,4,True
702,0xb1b7586656116d546033e3baff69bfcd6592225e,True,True,False,False,False,True,13,0,86,True,3,0,False,False,True,True,False,4,True
158,0x293c173b0ae90cd27c349cd8892efa625a98dbae,False,False,False,False,False,True,0,1,3,False,0,0,True,True,True,False,False,4,True


In [73]:
flag_bool =['has_interaction_toxic',
 'has_interaction_airdrop_m',
 'has_interaction_tornado',
 'has_interaction_disperse',
 'is_airdrop_master',
 'less_5_tx']

In [74]:
df_matching_address['flagged'] = df_matching_address[flag_bool].sum(axis=1) > 0
print(f'Number of flagged addresses: {df_matching_address["flagged"].sum()} out of {df_matching_address.shape[0]}')


Number of flagged addresses: 433 out of 1022


In [79]:
df_matching_address.to_csv('../suspicious_addresses.csv', index=False)

Almost half of the addresses are flagged

# Conclusion

We have found a few suspicious projects and addresses. 

The proposed stamps for gitcoin passport is passed by 10% of the wallets only. This means it may be a good indicator of human behavior. However it is not enough to flag a wallet as suspicious if it does not have transaction with a decentralized exchange.

The labels retrieved from Flipside Crypto are very useful and points to suspicious addresses that are colluding. They interact between themselves and with other suspicious addresses. This is a very good indicator of sybil behavior. And in particular the flag airdrop master. This addresses could be squelched.

Some labels such as tornado cash is less usefull. 

In the contribuotrs 20% had less than 5 transactions that is a lot of wallets.

